In [ ]:
import pymongo
cli = pymongo.MongoClient()
db = cli.test
coll = db.restaurants

# Explain with and without indexes

In [ ]:
coll.drop_indexes()
q = coll.find({'name': 'Dunkin Donuts'})
plan = q.explain()
plan

In [ ]:
plan.keys()

In [ ]:
plan['queryPlanner']

In [ ]:
plan['executionStats']

In [ ]:
coll.create_index('name')

In [ ]:
q = coll.find({'name': 'Dunkin Donuts'})
plan = q.explain()
plan

In [ ]:
plan['queryPlanner']

In [ ]:
input_stage = plan['queryPlanner']['winningPlan'].pop('inputStage')

In [ ]:
input_stage

In [ ]:
plan['queryPlanner']['winningPlan']

In [ ]:
from pprint import pprint
import copy
def plan_summary(plan):
    stages = []
    stage = copy.deepcopy(plan['queryPlanner']['winningPlan'])
    while stage:
        inputStage = stage.pop('inputStage', None)
        stages.append(stage)
        stage = inputStage
    stages.reverse()
    print('Execution Stages')
    for stage in stages:
        pprint(stage)
        print('--')

In [ ]:
plan = q.explain()
plan_summary(plan)

In [ ]:
def plan_stats(plan):
    stage = copy.deepcopy(plan['executionStats']['executionStages'])
    stages = []
    while stage:
        inputStage = stage.pop('inputStage', None)
        stages.append(stage)
        stage = inputStage
    stages.reverse()
    print('Execution Stats')
    for stage in stages:
        pprint(stage)
        print('--')

In [ ]:
plan_stats(plan)

# Removing the FETCH stage

In [ ]:
q = coll.find({'name': 'Dunkin Donuts'}, {'_id': 0, 'name': 1})
plan = q.explain()

In [ ]:
plan_summary(plan)

In [ ]:
plan_stats(plan)

## Index intersection

In [ ]:
coll.drop_indexes()
coll.create_index('borough')
coll.create_index('cuisine')

In [ ]:
q = coll.find({
    'borough': 'Queens',
    'cuisine': 'Bakery'
})
plan = q.explain()
plan_summary(plan)

In [ ]:
plan_stats(plan)

# Sorting with Explain

In [ ]:
coll.drop_indexes()

In [ ]:
coll.create_index('address.zipcode')

In [ ]:
doc = coll.find_one()

In [ ]:
q = coll.find({'address.zipcode': doc['address']['zipcode']})
plan = q.explain()
plan_summary(plan)

In [ ]:
q = q.sort('name')
plan = q.explain()
plan_summary(plan)

In [ ]:
coll.create_index([
    ('address.zipcode', 1),
    ('name', 1)])

In [ ]:
plan = q.explain()
plan_summary(plan)

In [ ]:
q = q.sort('name', -1)
plan = q.explain()
plan_summary(plan)

In [ ]:
coll.drop_indexes()
coll.create_index('address.zipcode')
coll.create_index('name')

In [ ]:
plan = q.explain()
plan_summary(plan)

In [ ]:
doc

# Geospatial indexing

In [ ]:
coll.drop_indexes()
coll.create_index([
    ('address.coord', '2dsphere')])
q = coll.find(
    {'address.coord': {
        '$nearSphere': doc['address']['coord']}},
    limit=5)
plan = q.explain()
plan_summary(plan)

In [ ]:
def print_stage_tree(stage, depth=0):
    indent = '  ' * depth
    print('{}{}'.format(indent, stage['stage']))
    if 'inputStage' in stage:
        print_stage_tree(stage['inputStage'], depth + 1)
    elif 'inputStages' in stage:
        for istage in stage['inputStages']:
            print_stage_tree(istage, depth + 1)

In [ ]:
print_stage_tree(plan['queryPlanner']['winningPlan'])

In [ ]:
plan['queryPlanner']['winningPlan']['inputStage']['inputStages'][0]['inputStage']

# Full-text indexing

In [ ]:
coll.drop_indexes()
coll.create_index([('name', 'text')])

In [ ]:
q = coll.find({'$text': {
    '$search': 'dunkin',
    '$caseSensitive': False
}})
plan = q.explain()
plan_summary(plan)

In [ ]:
plan_stats(plan)

In [ ]:
coll.drop_indexes()
coll.create_index([
    ('cuisine', 1),
    ('name', 'text')])

In [ ]:
q = coll.find(
    {'$text': {
        '$search': 'dunkin',
        '$caseSensitive': False},
     'cuisine': 'Donuts'})
plan = q.explain()
plan_summary(plan)

In [ ]:
plan_stats(plan)

In [ ]:
q.count()